In [79]:
import pandas as pd
import os
import string
from nltk.stem import WordNetLemmatizer as wn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder as le

In [153]:
csv_path = '../Downloads/fake_real_news'
newsdf = pd.read_csv(os.path.join(csv_path, 'clean_data_True.csv'))

In [81]:
newsdf['full'] = newsdf['title'] + ' ' + newsdf['text'] # create a new column with full text
newsdf['label'] = 1
newsdf.drop (["subject", 'date', 'title', 'text'], axis = 1, inplace=True)

In [154]:
newsdf

,Unnamed: 0,text,target
0,0,budget fight loom republican flip fiscal scrip...,0
1,1,military accept transgender recruit monday pen...,0
2,2,senior republican senator let mueller job wash...,0
3,3,fbi russia probe help australian diplomat tip ...,0
4,4,trump want postal service charge much amazon s...,0
...,...,...,...
21412,21412,fully commit nato back new approach afghanista...,0
21413,21413,lexisnexis withdraw two product chinese market...,0
21414,21414,minsk cultural hub become from authority minsk...,0
21415,21415,vatican upbeat possibility pope francis visit ...,0


In [155]:
clean_data = newsdf[['text', 'target']]
clean_data

,text,target
0,budget fight loom republican flip fiscal scrip...,0
1,military accept transgender recruit monday pen...,0
2,senior republican senator let mueller job wash...,0
3,fbi russia probe help australian diplomat tip ...,0
4,trump want postal service charge much amazon s...,0
...,...,...
21412,fully commit nato back new approach afghanista...,0
21413,lexisnexis withdraw two product chinese market...,0
21414,minsk cultural hub become from authority minsk...,0
21415,vatican upbeat possibility pope francis visit ...,0


In [83]:
# Dividing in X and y
X = newsdf['full']
y = newsdf['label']

In [84]:
def lematize (words):
    for index, word in enumerate (words):
        words[index] = wn().lemmatize(word, pos='v')
    for index, word in enumerate (words):
        words[index] = wn().lemmatize(word, pos='r')
    for index, word in enumerate (words):
        words[index] = wn().lemmatize(word, pos='a')
    for index, word in enumerate (words):
        words[index] = wn().lemmatize(word, pos='n')
    for index, word in enumerate (words):
        words[index] = wn().lemmatize(word, pos='s')
    return ' '.join(words)

In [85]:
stop_words = stopwords.words('english')

In [89]:
def preprocessing(sentence):
    sentence = sentence.strip()
    
    sentence = sentence.lower()
    
    sentence = ''.join(char for char in sentence if not char.isdigit())
    
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, ' ') 
    
    sentence = sentence.strip()
    
    sentence = word_tokenize (sentence)
    
    for word in sentence:
        if word in stop_words:
            sentence.remove(word)
        if len(word) < 3:
            if word in sentence:
                sentence.remove(word)
            
    lematize(sentence)
            
    return ' '.join(sentence)

In [90]:
X = X.str.replace ("’", "'", regex = False)
X = X.str.replace ("‘", "'", regex = False)
X = X.str.replace ('“', '"', regex = False)
X = X.str.replace ('”', '"', regex = False)

In [91]:
X_proc = X.apply(preprocessing)

In [92]:
X_proc # to see final product of preprocessing

0        budget fight loom republican flip fiscal scrip...
1        military accept transgender recruit monday pen...
2        senior republican senator let mueller job wash...
3        fbi russia probe help australian diplomat tip ...
4        trump want postal service charge much amazon s...
                               ...                        
21412    fully commit nato back new approach afghanista...
21413    lexisnexis withdraw two product chinese market...
21414    minsk cultural hub become from authority minsk...
21415    vatican upbeat possibility pope francis visit ...
21416    indonesia buy billion worth russian jet jakart...
Name: full, Length: 21417, dtype: object

In [93]:
y_enc = pd.Series(le().fit_transform(y))

In [94]:
y_enc # to see final product of encoding

0        0
1        0
2        0
3        0
4        0
        ..
21412    0
21413    0
21414    0
21415    0
21416    0
Length: 21417, dtype: int64

In [95]:
clean_data = pd.concat((X_proc, y_enc), axis=1)
clean_data.columns = ['text', 'target']
clean_data

,text,target
0,budget fight loom republican flip fiscal scrip...,0
1,military accept transgender recruit monday pen...,0
2,senior republican senator let mueller job wash...,0
3,fbi russia probe help australian diplomat tip ...,0
4,trump want postal service charge much amazon s...,0
...,...,...
21412,fully commit nato back new approach afghanista...,0
21413,lexisnexis withdraw two product chinese market...,0
21414,minsk cultural hub become from authority minsk...,0
21415,vatican upbeat possibility pope francis visit ...,0


In [156]:
clean_data

,text,target
0,budget fight loom republican flip fiscal scrip...,0
1,military accept transgender recruit monday pen...,0
2,senior republican senator let mueller job wash...,0
3,fbi russia probe help australian diplomat tip ...,0
4,trump want postal service charge much amazon s...,0
...,...,...
21412,fully commit nato back new approach afghanista...,0
21413,lexisnexis withdraw two product chinese market...,0
21414,minsk cultural hub become from authority minsk...,0
21415,vatican upbeat possibility pope francis visit ...,0


In [159]:
clean_data.to_csv(os.path.join(csv_path, 'clean_data_True.csv'), index=False)